References:  
https://www.kaggle.com/code/owaistahir/lstm-sentiment-analysis/input?select=IMDB+Dataset.csv

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D

2023-06-20 14:35:41.494489: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-20 14:35:41.562752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 14:35:42.503937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

maxlen = 400
batch_size = 128
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10
n_samples=50000

In [3]:
dataset = pd.read_csv('data/IMDB Dataset.csv')
print(dataset.shape)
dataset['sentiment'] = dataset['sentiment'].replace({'positive': 1, 'negative': 0})
# sample only 5000 rows randomly (full dataset leads to memory issues)
dataset = dataset.sample(n=n_samples, random_state=42)
dataset

(50000, 2)


,review,sentiment
33553,I really liked this Summerslam due to the look...,1
9427,Not many television shows appeal to quite as m...,1
199,The film quickly gets to a major chase scene w...,0
12447,Jane Austen would definitely approve of this o...,1
39489,Expectations were somewhat high for me when I ...,0
...,...,...
11284,`Shadow Magic' recaptures the joy and amazemen...,1
44732,I found this movie to be quite enjoyable and f...,1
38158,Avoid this one! It is a terrible movie. So wha...,0
860,This production was quite a surprise for me. I...,1


In [4]:
dataset = dataset.to_numpy()
dataset[0]

array(["I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna defe

In [5]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[0])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
                
            except KeyError:
                pass # no matching token in the Google w2v vocab
            
        vectorized_data.append(sample_vecs)
    
    return vectorized_data

In [7]:
def collect_expected(dataset):
    """Peel off the target values from the dataset"""
    expected = []
    for sample in dataset:
        expected.append(sample[1])
    return expected

In [8]:
#pass data into the functions
vectorized_data = tokenize_and_vectorize(dataset)

In [9]:
expected = collect_expected(dataset)

In [10]:
#train/test split
split_point = int(len(vectorized_data)*.8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]

x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [11]:
#truncate any review longer than 400 tokens
#pad the shorted smaples out to 400 tokens with null or 0


def pad_trunc(data, maxlen):
    """
    for a given dataset pad with zero vectors or truncate to maxlen
    """
    new_data = []
    
    #create a vector of 0s the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
        
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            #append the appropriate number 0 vectors to the list
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)         
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [12]:
#pass the train and test data to padder/truncator
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

In [13]:
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)

x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [14]:
num_neurons = 50
model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

2023-06-20 14:43:36.143854: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 400, 50)           70200     
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 1)                 20001     
                                                                 
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________


2023-06-20 14:43:38.208224: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 14:43:38.210089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 14:43:38.211428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:
#fit model
model.fit(x_train, y_train,
         batch_size = batch_size,
         epochs=epochs,
         validation_data=(x_test, y_test))


Epoch 1/10


2023-06-20 14:45:55.718284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 14:45:55.720522: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 14:45:55.722165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

313/313 [==============================] - ETA: 0s - loss: 0.4996 - accuracy: 0.7602

2023-06-20 14:47:45.958104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 14:47:45.960039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 14:47:45.961398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

313/313 [==============================] - 122s 381ms/step - loss: 0.4996 - accuracy: 0.7602 - val_loss: 0.3929 - val_accuracy: 0.8309
Epoch 2/10
313/313 [==============================] - 105s 335ms/step - loss: 0.3768 - accuracy: 0.8357 - val_loss: 0.4309 - val_accuracy: 0.8074
Epoch 3/10
313/313 [==============================] - 104s 331ms/step - loss: 0.3268 - accuracy: 0.8611 - val_loss: 0.3220 - val_accuracy: 0.8646
Epoch 4/10
313/313 [==============================] - 103s 329ms/step - loss: 0.2946 - accuracy: 0.8782 - val_loss: 0.3149 - val_accuracy: 0.8710
Epoch 5/10
313/313 [==============================] - 101s 324ms/step - loss: 0.2711 - accuracy: 0.8892 - val_loss: 0.3160 - val_accuracy: 0.8717
Epoch 6/10
313/313 [==============================] - 100s 320ms/step - loss: 0.2491 - accuracy: 0.8975 - val_loss: 0.3386 - val_accuracy: 0.8565
Epoch 7/10
313/313 [==============================] - 101s 324ms/step - loss: 0.2302 - accuracy: 0.9080 - val_loss: 0.3715 - val_accura

In [17]:
model.save('models/w2v_lstm_amazonreviews')


2023-06-20 15:08:08.050925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 15:08:08.052658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 15:08:08.054060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: models/w2v_lstm_amazonreviews/assets


INFO:tensorflow:Assets written to: models/w2v_lstm_amazonreviews/assets
